Source: https://github.com/oschow/take-a-hike/blob/master/AllTrails/scrape_clean/scrape_ratings.py

In [1]:
# installs
!pip install selenium
!pip install wget
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

     |████████████████████████████████| 911kB 18.2MB/s 
  Created wheel for wget: filename=wget-3.2-cp37-none-any.whl size=9681 sha256=32083e5aa067a1d06f2f38c74c0cf4168b28e2fb932ccb1a337d5b857eb3f823
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubun

In [2]:
# imports
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup, SoupStrainer
import time, sys, re, urllib, csv, wget, requests

In [ ]:
# Set up chromedriver
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: use options instead of chrome_options
  import sys


In [ ]:
# Definition for logging in
def login(browser):
    browser.get('https://www.alltrails.com/login?ref=header')
    username = browser.find_element_by_name("userEmail")
    password = browser.find_element_by_name("userPassword")
    username.send_keys("221ornithomimus@gmail.com")
    password.send_keys("Dj3!120423u_9jkd_5!k34")
    browser.find_element_by_xpath("//input[@data-test-id='formButton-submit']").click()


# Scroll through the page and load all of the hikes
def get_hikes(browser, difficulty):
    if (difficulty != 'none'):
        browser.get('https://www.alltrails.com/canada/british-columbia')
        time.sleep(2)
        elements = browser.find_elements_by_xpath("//img[@alt='Caret']") # find the difficulty list
        time.sleep(2)
        elements[0].click() #click the difficulty list
        time.sleep(2)
        if (difficulty == 'easy'):
            difficulty_box = browser.find_element_by_xpath("//input[@value='1,2']") # find the difficulty checkboxes
            difficulty_box.click()
            if (difficulty_box.is_selected()): print("Selected: Easy")
            else: print("Error selecting difficulty")
        elif (difficulty == 'moderate'):
            difficulty_box = browser.find_element_by_xpath("//input[@value='3,4']")
            difficulty_box.click()
            if (difficulty_box.is_selected()): print("Selected: Moderate")
            else: print("Error selecting difficulty")
        elif (difficulty == 'hard'):
            difficulty_box = browser.find_element_by_xpath("//input[@value='5,6,7,8']")
            difficulty_box.click()
            if (difficulty_box.is_selected()): print("Selected: Hard")
            else: print("Error selecting difficulty")

    time.sleep(20)
    i = 0
    while True:
        try:
            element = WebDriverWait(browser, 15).until(EC.presence_of_element_located((By.CSS_SELECTOR,"[title^= 'Show more trails']" )))
            element.click()
            time.sleep(7)
            print("Getting hikes:", difficulty, str(i))
            i += 1
        except:
            break
    soup = BeautifulSoup(browser.page_source)
    return soup

# Get links to a bunch of photos
def get_photo_links(hike, browser):
    browser.get(hike[0])
    soup2 = BeautifulSoup(browser.page_source)
    hikes = soup2.find_all(class_="trail-photo lazyloaded")[:3] # get first 3 photos
    i = 0
    links = []
    for element in hikes:
        hike_url = element.get('src') # get url from hike
        links += [(hike_url, hike[1] + "-" + str(i))]
        i += 1
        time.sleep(1) # wait 1 second to avoid being blocked
    return links

In [ ]:
# Log in to AllTrails
login(wd)

In [ ]:
# Scroll through and load all available easy hikes
easy_soup = get_hikes(wd, 'easy')
easy_hikes = easy_soup.find_all(itemid=re.compile('/trail/\S*'))

In [ ]:
# Get the hike urls for easy hikes
easy_hike_links = []

for hike in easy_hikes:
    tag = hike.get('itemid').split('/')[-1]
    easy_hike_links += [['https://www.alltrails.com' + hike.get('itemid') + '/photos', tag]]

# Save hike links to a csv file for backup
with open('easy_hike_links_3-29.csv', mode='w') as f:
    writer = csv.writer(f)
    writer.writerows(easy_hike_links)

In [ ]:
# Scroll through all moderate hikes
moderate_soup = get_hikes(wd, 'moderate')
moderate_hikes = moderate_soup.find_all(itemid=re.compile('/trail/\S*'))

In [ ]:
# Get the hike urls for moderate hikes
moderate_hike_links = []

for hike in moderate_hikes:
    tag = hike.get('itemid').split('/')[-1]
    moderate_hike_links += [['https://www.alltrails.com' + hike.get('itemid') + '/photos', tag]]

# Save hike links to a csv file for backup
with open('moderate_hike_links_3-29.csv', mode='w') as f:
    writer = csv.writer(f)
    writer.writerows(moderate_hike_links)

In [ ]:
# Get all photo links for easy hikes and save them to a csv file
easy_photo_links = []
for link in easy_hike_links:
    easy_photo_links += get_photo_links(link, wd)

# Save photo links to a csv file
with open('easy_photo_links_3-29.csv', mode='w') as f:
    writer = csv.writer(f)
    writer.writerows(easy_photo_links)

In [ ]:
# Get all photo links for moderate hikes and save them to a csv file
moderate_photo_links = []
for link in moderate_hike_links:
    moderate_photo_links += get_photo_links(link, wd)

# Save photo links to a csv file
with open('moderate_photo_links_3-29.csv', mode='w') as f:
    writer = csv.writer(f)
    writer.writerows(moderate_photo_links)

In [ ]:
# Scroll through all hard hikes

hard_soup = get_hikes(wd, 'hard')
hard_hikes = hard_soup.find_all(itemid=re.compile('/trail/\S*'))

In [ ]:
# Get a list of hike urls and their id numbers
hard_hike_links = []

for hike in hard_hikes:
    tag = hike.get('itemid').split('/')[-1]
    hard_hike_links += [['https://www.alltrails.com' + hike.get('itemid') + '/photos', tag]]

# Save hike links to a csv file for backup
with open('hard_hike_links_3-29.csv', mode='w') as f:
    writer = csv.writer(f)
    writer.writerows(hard_hike_links)

In [ ]:
# Get all photo links for hard hikes and save them to a csv file
hard_photo_links = []
for link in hard_hike_links:
    hard_photo_links += get_photo_links(link, wd)

# Save photo links to a csv file
with open('hard_photo_links_3-29.csv', mode='w') as f:
    writer = csv.writer(f)
    writer.writerows(hard_photo_links)

Once we have the photo urls saved to a csv file, we can start downloading. Make sure you have uploaded one of the following files:
- easy_photo_links_3-29.csv
- moderate_photo_links_3-29.csv
- hard_photo_links_3-29.csv

Next, specify the directory you want the photos saved to and the range of photos to download. To download the entire csv, omit the range.

Finally, specify the name of the zip folder for downloading the images.

In [3]:
# Read csv file and save it as a 2D array, where photo_links[i] is a tuple of [url, label]
my_photo_links = []

with open("easy_photo_links_3-29.csv") as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        my_photo_links.append(row)

In [4]:
print(len(my_photo_links))

2991


In [5]:
# Download a range of photos and save them in the directory specified
!mkdir photos5
root_dir = "/content/photos5/"

i = 0

for entry in my_photo_links[2000:len(my_photo_links)]:
    # Check for invalid photo urls
    if (entry[0][0] == '/'):
        continue
    with requests.get(entry[0], stream=True) as response:
        filename = root_dir + entry[1] + ".jpg"
        with open(filename, "wb") as f:
            for chunk in response.iter_content(chunk_size=1024):
                if chunk:
                    f.write(chunk)
    time.sleep(10)
    if (i%50 == 0):
        print("i =",str(i))
    i += 1

i = 0
i = 50
i = 100
i = 150
i = 200
i = 250
i = 300
i = 350
i = 400
i = 450
i = 500
i = 550
i = 600
i = 650
i = 700
i = 750
i = 800
i = 850
i = 900
i = 950


In [6]:
# Zip your photos for downloading and saving
!zip -r easy_photos5.zip photos5

  adding: photos5/ (stored 0%)
  adding: photos5/coast-trail-from-point-to-eagle-cove-2.jpg (deflated 0%)
  adding: photos5/richmond-nature-park-east-loop-0.jpg (deflated 0%)
  adding: photos5/south-arm-marshes-spirit-trail-1.jpg (deflated 0%)
  adding: photos5/brockton-point-trail-0.jpg (deflated 0%)
  adding: photos5/ward-s-ferry-trail-1.jpg (deflated 0%)
  adding: photos5/maud-island-trail-quadra-island-0.jpg (deflated 0%)
  adding: photos5/reservoir-doc-findlay-and-kinnickinnick-loop-0.jpg (deflated 0%)
  adding: photos5/railway-path-2.jpg (deflated 0%)
  adding: photos5/channel-ridge-bench-trail-1.jpg (deflated 0%)
  adding: photos5/pine-tree-trail-2.jpg (deflated 0%)
  adding: photos5/gallagher-hill-trail-1.jpg (deflated 0%)
  adding: photos5/semiahmoo-heritage-south-trail-0.jpg (deflated 0%)
  adding: photos5/butterfly-and-mcivers-trail-loop-2.jpg (deflated 0%)
  adding: photos5/bright-angel-loop-1.jpg (deflated 0%)
  adding: photos5/north-arm-bridge-bikeway-2.jpg (deflated 0%)


VVV _____ Here be dragons _____ VVV

In [ ]:
wd.get('https://www.alltrails.com/canada/british-columbia')
time.sleep(2)
difficulties = wd.find_elements_by_xpath("//input[@type='checkbox']")
print(len(difficulties))
elements = wd.find_elements_by_xpath("//img[@alt='Caret']") # find the difficulty list
time.sleep(2)
elements[0].click() #click the difficulty list
time.sleep(5)
# element = WebDriverWait(browser, 15).until(EC.presence_of_element_located((By.CSS_SELECTOR,"[title^= 'Show more trails']" )))
difficulty = wd.find_element_by_xpath("//input[@value='5,6,7,8']") # find the difficulty checkboxes
print(difficulty.is_selected())
difficulty.click() # Click the checkbox matching the difficulty
print(difficulty.is_selected())
time.sleep(20)
element = WebDriverWait(wd, 15).until(EC.presence_of_element_located((By.CSS_SELECTOR,"[title^= 'Show more trails']" )))
element.click()
time.sleep(7)

soup = BeautifulSoup(wd.page_source)
debug_hikes = soup.find_all(itemid=re.compile('/trail/\S*'))
tag = debug_hikes[10].get('itemid').split('/')[-1]
print(tag)

print(debug_hikes)

In [ ]:
# Select difficulty
wd.get('https://www.alltrails.com/canada/british-columbia')
time.sleep(1)
elements = wd.find_elements_by_xpath("//img[@alt='Caret']")
# browser.find_element_by_xpath("//input[@data-test-id='formButton-submit']").click()
# driver.find_element_by_xpath("//input[starts-with (@name,'Tut')]")
# xpath=//div[matches(@id,'che.*boxes')]
time.sleep(2)
#print(elements[0])
elements[0].click()
time.sleep(2)
easyButton = wd.find_elements_by_xpath("//input[@type='checkbox']")

In [ ]:
easyButton[0].click()

In [ ]:
# Download each image in the list
for hike in hike_links:
    get_photos(hike,wd)

In [ ]:
wd.get(hike_links[0][0])
soup2 = BeautifulSoup(wd.page_source)
# hikes = soup2.find_all(class='photo-item gallery sm')

In [ ]:
photos = soup2.find_all(class_="photo-item gallery sm")[:5]

In [ ]:
photos

In [ ]:
# Go to page
wd.get("https://www.alltrails.com/canada/british-columbia")

# For each entry, get itemid
soup = BeautifulSoup(wd.page_source)

hikes = soup.select('div data-hydrate')

In [ ]:
hikes = soup.select('div data-hydrate')

In [ ]:
hikes

[]